In [25]:
pip install rouge

In [26]:
import json
import joblib
import math
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from google.colab import drive
from collections import Counter
from rouge import Rouge

In [27]:
# Mount Google Drive
drive.mount('/content/drive')

# Specify the correct file path
file_path = '/content/drive/MyDrive/Colab Notebooks/text.json'

# Load and view JSON data
with open(file_path, 'r') as json_file:
    json_data = json.load(json_file)

# Display the JSON data
print(json_data)

# Display the first few records
num_records_to_display = 5
for i, record in enumerate(json_data[:num_records_to_display]):
    print(f"Record {i + 1}: {record}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [28]:
# Function to summarize text
def summarize_text(text):
    sentences = text.split('.')
    words = [word.lower() for sentence in sentences for word in sentence.split()]
    word_counts = Counter(words)

    # Calculate the TF-IDF score for each word in the text
    word_scores = {}
    for word, count in word_counts.items():
        # Use TF-IDF to calculate the weight of each word
        tf_idf = count / len(words) * math.log(len(sentences) / (1 + word_counts[word]), 10)

        # Add the weighted word frequency to the sentence score
        word_scores[word] = tf_idf

    # Rank sentences based on the sum of their word scores
    sentence_scores = {}
    for i, sentence in enumerate(sentences):
        sentence_scores[i] = sum(word_scores.get(word, 0) for word in sentence.split())

    # Get the indices of the top-ranked sentences
    top_sentences = sorted(sentence_scores, key=sentence_scores.get, reverse=True)[:5]

    # Extract and return the top-ranked sentences as the summary
    summary = ' '.join(sentences[i] for i in top_sentences)
    return summary

# Summarize the text for the first record
print(summarize_text(json_data[0]['text']))

 This routing was extended by and labelled as M-137 on maps in 1931  M-137 was maintained by MDOT like other state highways in Michigan  MDOT announced on August 6, 2020, that jurisdiction had been transferred at the beginning of June and that all M-137 signage had since been removed  ==Major intersections== ==See also== * ==References== ==External links== *Former M-137 at Michigan Highways 137 Category:Transportation in Grand Traverse County, Michigan   The highway was first shown without a number label on maps in 1930 and labeled after an extension the next year


In [31]:
# Summarize the text for the first record
if 'text' in json_data[0]:
    text_summary = summarize_text(json_data[0]['text'])
    print("Text Summary:")
    print(text_summary)

    # Optionally, you can compare the summary with a reference summary or use other evaluation metrics.
    # For example, if you have a reference summary in your data:
    reference_summary = json_data[0]['text']

    # Calculate ROUGE scores
    rouge = Rouge()
    rouge_scores = rouge.get_scores(text_summary, reference_summary)
    print("ROUGE Scores:")
    print(rouge_scores)
else:
    print("No 'text' key found in the first record.")

Text Summary:
 This routing was extended by and labelled as M-137 on maps in 1931  M-137 was maintained by MDOT like other state highways in Michigan  MDOT announced on August 6, 2020, that jurisdiction had been transferred at the beginning of June and that all M-137 signage had since been removed  ==Major intersections== ==See also== * ==References== ==External links== *Former M-137 at Michigan Highways 137 Category:Transportation in Grand Traverse County, Michigan   The highway was first shown without a number label on maps in 1930 and labeled after an extension the next year
ROUGE Scores:
[{'rouge-1': {'r': 0.29694323144104806, 'p': 1.0, 'f': 0.4579124543817525}, 'rouge-2': {'r': 0.19534883720930232, 'p': 0.9655172413793104, 'f': 0.324951641301363}, 'rouge-l': {'r': 0.29694323144104806, 'p': 1.0, 'f': 0.4579124543817525}}]
